In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [ ]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [ ]:
val_df['Image_ID'].nunique()

In [ ]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("runs/detect/train*/weights/best.pt"))[-1]
BEST_PATH

In [ ]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

model = model.eval()

In [ ]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image(filepath):
    image = Image.open(filepath)

    exif = image._getexif()
    if exif is None:
        return image
        # return torch.tensor(np.array(image.convert("RGB")))

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image
    # return torch.tensor(np.array(image.convert("RGB")))


flag

In [ ]:
# Validate the model on the validation set
BEST_CFG = sorted(glob("runs/detect/train*/args.yaml"))[-1]
BEST_CFG

In [ ]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = [i for i in os.listdir(test_dir_path) if not i.endswith(".npy")]

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Batch size for predictions
batch_size = 16

# Process images in batches
for i in tqdm(range(0, len(image_files), batch_size)):
	batch_files = image_files[i:i + batch_size]
	batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

	# Make predictions on the batch of images
	results = model.predict(
		batch_images,
		conf=0.,
		imgsz=1024,
		batch=batch_size,
		verbose=False,
		device="cuda:0",

		seed=0,
		deterministic=True,
		close_mosaic=10,
		mask_ratio=4,
		iou=.7,
		max_det=100,
		format="torchscript",
		nms=True,
		simplify=True,
		vid_stride=1,
		cfg=BEST_CFG,
	)

	# Iterate through each result in the batch
	for img_file, result in zip(batch_files, results):
		boxes = result.boxes.xyxy.tolist() if result.boxes else []  # Bounding boxes in xyxy format
		classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
		confidences = result.boxes.conf.tolist() if result.boxes else []  # Confidence scores
		names = result.names  # Class names dictionary

		if boxes:  # If detections are found
			for box, cls, conf in zip(boxes, classes, confidences):
				x1, y1, x2, y2 = box
				detected_class = names[int(cls)]  # Get the class name from the names dictionary

				# Add the result to the all_data list
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": detected_class,
						"confidence": conf,
						"ymin": y1,
						"xmin": x1,
						"ymax": y2,
						"xmax": x2,
					}
				)
		else:  # If no objects are detected
			all_data.append(
				{
					"Image_ID": str(img_file),
					"class": "None",
					"confidence": None,
					"ymin": None,
					"xmin": None,
					"ymax": None,
					"xmax": None,
				}
			)


In [ ]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [ ]:
sub.head()

In [ ]:
sub.describe()

In [ ]:
sub['class'].value_counts()

In [ ]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)
	paths = [i for i in label_folder.glob("*") if i.suffix != ".npy"]

	for label_file in paths:
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []

	paths = [i for i in image_folder.glob("*") if i.suffix != ".npy"]
	for image_file in paths:
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = [i for i in image_folder.glob(f"{row['Image_ID']}*") if i.suffix != ".npy"]
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

In [ ]:
converted_labels.describe()

In [ ]:
converted_labels["Image_ID"].nunique()

In [ ]:
converted_labels['class_id'].value_counts()

In [ ]:
class_map

In [ ]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

In [ ]:
converted_labels.sample(5)

In [ ]:
sub.sample(5)

In [ ]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

In [ ]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [ ]:
converted_labels.isna().sum()

In [ ]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

In [ ]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [ ]:
sub.sample(3)

In [ ]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

In [ ]:
predictions = get_preds_data(sub, None)

len(predictions)

In [ ]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

In [ ]:
converted_labels.sample(5)

In [ ]:
import torch
from torchmetrics.detection import MeanAveragePrecision

def compute_map(preds, targets):
	"""
	Compute mAP at different IoU thresholds using torchmetrics.
	
	Args:
		preds: List of dicts with 'boxes', 'scores', 'labels' for predictions
		targets: List of dicts with 'boxes', 'labels' for ground truth
	
	Returns:
		Dict containing mAP results for each IoU threshold
	"""
	# Initialize the metric
	metric = MeanAveragePrecision()
	
	# Update metric with predictions and targets
	metric.update(preds, targets)
	
	# Compute the results
	result = metric.compute()
	
	return result

thrs = np.linspace(0.0001, 0.95, 15)
# Example usage
# if __name__ == "__main__":
# Example predictions and targets
# iou_thresholds = [0.5]
for i in thrs:
	preds = list(get_preds_data(sub, i).values())

	targets = list(ground_truth.values())

	# Compute mAP
	results = compute_map(preds, targets)

	# Print results
	print("mAP Results:", i, " - ", results)

In [ ]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [ ]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)